In [2]:
#!pip install simplejson

In [3]:
# Import dependencis
import os, tweepy
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError
import csv,ssl,time
import warnings
warnings.filterwarnings('ignore') # Hides warning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)

In [4]:
# API credentials - need to move them to config file 
consumerKey ="8QzKAL9poBEwy0vgD2RMXKTDv"
consumerSecretKey = "rJHM1691uU6zk9ulFehVQznpEgmpZUfd3HVE7VMMlTR2KAJjA3"
accessToken = "1247938131841683456-oSN2nufp1b0tUZkYqe3kUs6j3VhisL"
accessToeknSecret= "9a7EbDW1jww5KQiR6PxEsaf0aaySh9ZE37lgpuPgbpTZY"

In [5]:
# File paths
tweet_data_file = os.path.join("","Resources","data","tweetStreamSearch.csv")

In [6]:
# Authenticate twitter account
auth = tweepy.OAuthHandler(consumer_key= consumerKey,consumer_secret= consumerSecretKey)

In [7]:
# Set access token
auth.set_access_token(accessToken,accessToeknSecret)

In [8]:
# Create API instance with authentication
api= tweepy.API(auth,wait_on_rate_limit=True)

In [9]:
# Set keywords, hashtag or mentions tag and # of searches - keep in mind the daily limitation
keywords = '#Trump OR Trump OR Biden OR #Biden'
key_words =['Trump','#Trump','Biden','#Biden']

In [10]:
# Function for handling pagination in our search
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            print('Reached rate limite. Sleeping for >15 minutes')
            time.sleep(15 * 61)

In [17]:
class StreamListener(tweepy.StreamListener):

    def on_status(self, status):

        try:
            # saves the tweet object
            tweet_object = status

            # Checks if its a extended tweet (>140 characters)
            if 'extended_tweet' in tweet_object._json:
                tweet = tweet_object.extended_tweet['full_text']
            else:
                tweet = tweet_object.text

            # Convert all named and numeric character references (e.g. &gt;, &#62;, &#x3e;) in the string 
            # to the corresponding Unicode characters'''
            tweet = (tweet.replace('&amp;', '&').replace('&lt;', '<')
                     .replace('&gt;', '>').replace('&quot;', '"')
                     .replace('&#39;', "'").replace(';', " ")
                     .replace(r'\u', " "))

            # Remove special characters, numbers, punctuations
            tweet=tweet.replace("[^a-zA-Z#]", " ")
            
            # Save the keyword that matches the stream
            keyword_matches = []
            for word in key_words:
                if word.lower() in tweet.lower():
                    keyword_matches.extend([word])

            keywords_strings = ", ".join(str(x) for x in keyword_matches)
            
            if len(keywords_stringsstrip())>0:
                # Save other information from the tweet
                user = status.author.screen_name
                timeTweet = status.created_at
                source = status.source
                tweetId = status.id
                tweetUrl = "https://twitter.com/statuses/" + str(tweetId)
                followers= status.author.followers_count #Indicates user follower
                friends= status.author.friends_count #Friend Count
                favorite= status.favorite_count #Indicates approx how many times this Tweet has been liked by Twitter users

                # Exclude retweets, too many mentions and too many hashtags
                if not any((('RT @' in tweet, 'RT' in tweet,
                           tweet.count('@') >= 2, tweet.count('#') >= 3))):

                    # Saves the tweet information in a new row of the CSV file
                    writer.writerow([tweet,keywords_strings,timeTweet,user, source, 
                                     tweetId, tweetUrl,followers,friends,favorite])

        except StopIteration:
            time.sleep(1001)
            pass
        
        except Exception as e:
            print('Encountered Exception:', e)
            time.sleep(1001)
            pass



In [18]:
# Function to save tweets in to csv
def saveTweets():

    # Opening a CSV file to save the gathered tweets
    with open(tweet_data_file, 'w',encoding='utf-8', newline='') as file:
        global writer
        writer = csv.writer(file)

        # Add a header row to the CSV
        writer.writerow(["Tweet", "Matched Keywords", "Date","User","Source", 
                         "Tweet ID", "Tweet URL","Followers","Friends","Favorite"])


        # Initializing the twitter streap Stream
        try:
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(track=keywords)

        # Stop temporarily when hitting Twitter rate Limit
        except tweepy.RateLimitError:
            print("RateLimitError...waiting ~15 minutes to continue")
            time.sleep(1001)
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(track=[keywords])

        # Stop temporarily when getting a timeout or connection error
        except (Timeout, ssl.SSLError, ReadTimeoutError,
                ConnectionError) as exc:
            print("Timeout/connection error...waiting ~15 minutes to continue")
            time.sleep(1001)
            streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
            streamingAPI.filter(track=[keywords])

        # Stop temporarily when getting other errors
        except tweepy.TweepError as e:
            if 'Failed to send request:' in e.reason:
                print("Time out error caught.")
                time.sleep(1001)
                streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
                streamingAPI.filter(track=[keywords])
            else:
                print("Other error with this user...passing")
                time.sleep(1001)
                streamingAPI = tweepy.streaming.Stream(auth, StreamListener())
                streamingAPI.filter(track=[keywords])


In [20]:
#Get and Save Data
saveTweets()